### AUDIOVAULT: IMAGE IN AUDIO STEGANOGRAPHY

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PIL import Image
from utils import image_to_bits, check_bits_alteration
from embedding import embed_data
from extraction import extract_data
from config import TWO_BIT_QIM

In [3]:
#FILE PATHS
audio_path = "audio/blankspace.wav"
stego_path = "stego.wav"
secret_data_path = "images/cover.jpg"
extracted_data_path = "extracted.png"

In [4]:
secret_data, secret_shape = image_to_bits(path=secret_data_path)
files = [
    {
        "file_type":"jpg",
        "height":secret_shape[0],
        "width":secret_shape[1],
        "channels":secret_shape[2],
        "payload_length":len(secret_data)
    }
]

In [5]:
#Embed the secret data
embed_data(secret_data=secret_data,files=files,audio_path=audio_path,stego_path=stego_path)

HEADER BITS: ['0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0']
DECODED HEADER: {'version': 1, 'header_length': 112, 'file_count': 1, 'two_bit_qim': 1, 'files': [{'file_type': 'jpg', 'height': 853, 'width': 1280, 'channels': 3, 'payload_length': 26204160}]}
Frame Size: 7075179
TOTAL FRAMES: 2
Payload Capacity: 13265920 and Payload Length: 13102080
Payload Capacity: 11252530 and Payload Length: 13102080
Payload length (13102080) exceeds available valid embedding locations (11252530) in 

In [ ]:
#Extract the secret data
extracted_data = extract_data(audio_path=stego_path,extracted_data_path=extracted_data_path)

KeyboardInterrupt: 

In [7]:
check_bits_alteration(original=secret_data, extracted=extracted_data)

Mismatch location: 703174
Mismatch location: 703175
Mismatch location: 725121
Mismatch location: 873374
Mismatch location: 873379
Mismatch location: 873380
Mismatch location: 873381
Mismatch location: 873382
Mismatch location: 873384
Mismatch location: 873386
Mismatch location: 873387
Mismatch location: 873388
Mismatch location: 873389
Mismatch location: 873390
Mismatch location: 873391
Mismatch location: 873393
Mismatch location: 873394
Mismatch location: 873396
Mismatch location: 873400
Mismatch location: 873401
Mismatch location: 873402
Mismatch location: 873404
Mismatch location: 873405
Mismatch location: 873407
Mismatch location: 873409
Mismatch location: 873410
Mismatch location: 873411
Mismatch location: 873412
Mismatch location: 873413
Mismatch location: 873414
Mismatch location: 873415
Mismatch location: 873417
Mismatch location: 873419
Mismatch location: 873420
Mismatch location: 873422
Mismatch location: 873424
Mismatch location: 873425
Mismatch location: 873426
Mismatch loc